<a href="https://colab.research.google.com/github/gcosma/COP509/blob/main/LabSolutions/Lab_Exercise_(No_5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Lab Exercise (No. 5)**

**Task Overview**

Your task is to use the cleaned ArtReviews dataset and the ArtRatings(classes) and repeat the Lab Tutorial.

###***pre-work1: mount drive***

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


###***pre-work2: set datasets***

In [2]:
data_path = "/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/"
!ls "/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/"

# select datasets
datasets = 'Arts5000' # ArtsAll, Arts5000

if datasets == 'ArtsAll':
  # Arts all
  data_all_reviews = 'ArtsReviews.txt'
  data_all_ratings = 'ArtsRatings.txt'
  data_train_reviews = 'ArtsReviews_train.txt'
  data_test_reviews = 'ArtsReviews_test.txt'
  data_train_ratings = 'ArtsRatings_train.txt'
  data_test_ratings = 'ArtsRatings_test.txt'
else:
  # Arts 5000
  data_all_reviews = 'Reduced_ArtsReviews_5000.txt'
  data_all_ratings = 'Reduced_ArtsRatings_5000.txt'
  data_train_reviews = 'ArtsReviews_5000_train.txt'
  data_test_reviews = 'ArtsReviews_5000_test.txt'
  data_train_ratings = 'ArtsRatings_5000_train.txt'
  data_test_ratings = 'ArtsRatings_5000_test.txt'

ArtsRatings_5000_test.txt   ArtsReviews_5000_train.txt	Reduced_ArtsRatings_5000.txt
ArtsRatings_5000_train.txt  glove.6B.100d.txt		Reduced_ArtsReviews_5000.txt
ArtsReviews_5000_test.txt   metamorphosis_clean.txt	review_polarity


##**1. Installations**


In [3]:
!pip install transformers

##**2. Import the dataset**

In [4]:
from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
import random
import nltk
import numpy
import pandas as pd

nltk.download('stopwords')

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# convert to lower case
	tokens = [word.lower() for word in tokens]
	# remove punctuation from each token
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens

#Yan: not in the tutorial, students need to add.
# Write the code to read each line is a review
# To do this you need to load doc into memory as lines
def load_doc_lines(filename):
	 file = open(filename,'rt')
	 lines = list()
   # read all text as lines
	 while 1:

		 line = file.readline()
		 if not line:
		   break
		 pass
		 lines.append(line.strip("\n"))
   # close the file
	 file.close()
	 return lines

# load doc, clean and return line of tokens
def doc_to_line(doc):
	# load the doc
	# doc = load_doc(filename)
	# clean doc
	tokens = clean_doc(doc)
	return ' '.join(tokens)

# load all docs in a directory
def process_docs(files):
	lines = list()
	# walk through all files in the folder
	for doc in files:
		# print(len(doc))
		line = doc_to_line(doc)
		# add to list
		lines.append(line)
	return lines

# directly load train and test sets#############################################
ArRe_train_lines = load_doc_lines(data_path + data_train_reviews)
ArRa_train_lines = numpy.loadtxt(data_path + data_train_ratings, dtype = float)
ArRe_test_lines = load_doc_lines(data_path + data_test_reviews)
ArRa_test_lines = numpy.loadtxt(data_path + data_test_ratings, dtype = float)
# end directly load train and test sets#########################################

train_docs = ArRe_train_lines
test_docs = ArRe_test_lines

# process all train reviews
train_docs = process_docs(train_docs)

# process all test reviews
test_docs = process_docs(test_docs)

train_docs = pd.Series(train_docs)
test_docs = pd.Series(test_docs)

print(train_docs.shape)
print(test_docs.shape)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


(4000,)
(1000,)


##**3. Load the model**

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

# Loading the Pre-trained BERT model
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

##**4. Embed train set**

In [6]:
N = 500 # N number of train reviews as a group to be embeded
with torch.no_grad():
    # Tokenization
    tokenized = train_docs[0:N].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

    # padding
    max_len = 0
    q = 0
    for i in tokenized.values:

        # BERT only accepts  maximum 512 values
        if len(i) > 512:
            temp = tokenized.values[q]
            tokenized.values[q] = temp[:512]
            i = tokenized.values[q]
            print('too much tokenized.values for BERT, only 512 are taken')

        # print(len(i))
        if len(i) > max_len:
            max_len = len(i)
        q += 1

    padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
    np.array(padded).shape

    # masking
    attention_mask = np.where(padded != 0, 1, 0)
    attention_mask.shape

    # run the model
    input_ids = torch.tensor(padded)
    attention_mask = torch.tensor(attention_mask)

    print(input_ids.shape)

    last_hidden_states = model(input_ids, attention_mask=attention_mask)

    train_features = last_hidden_states[0][:,0,:].numpy()

print(len(train_features))


torch.Size([500, 350])
500


##**5. Embed test set**

In [7]:
N2 = 100 # N2 number of test reviews as a group to be embeded
with torch.no_grad():
    # Tokenization
    tokenized = test_docs[0:N2].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

    # padding
    max_len = 0
    q = 0
    for i in tokenized.values:

        # BERT only accept maximum 512 values
        if len(i) > 512:
            temp = tokenized.values[q]
            tokenized.values[q] = temp[:512]
            i = tokenized.values[q]
            print('too much tokenized.values for BERT, only 512 are taken')

        # print(len(i))
        if len(i) > max_len:
            max_len = len(i)
        q += 1

    padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
    np.array(padded).shape
    # masking
    attention_mask = np.where(padded != 0, 1, 0)
    attention_mask.shape
    # run the model
    input_ids = torch.tensor(padded)
    attention_mask = torch.tensor(attention_mask)

    print(input_ids.shape)

    last_hidden_states = model(input_ids, attention_mask=attention_mask)

    test_features = last_hidden_states[0][:,0,:].numpy()

print(len(test_features))

torch.Size([100, 223])
100


##**6. Evaluation**

In [8]:
train_labels = ArRa_train_lines[:N]
test_labels = ArRa_test_lines[:N2]

print(len(train_features))
print(len(train_labels))

lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

score = lr_clf.score(test_features, test_labels)
print(score)

from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

500
500
0.31
Dummy classifier score: 0.230 (+/- 0.00)
